In [1]:
%run uvms_base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# conditional function instances
# coupled forward dynamics of uvms
x_next_coupled, x_nextdd_coupled, states, u, dt, q_min, q_max, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=True)
x_next_coupled_eval = cs.Function('UVMSnext_coupled', [states, u, dt, vehicle_p, manipulator_p, base_T ,q_min, q_max], [x_next_coupled, u_checks, x_nextdd_coupled])


# decoupled forward dynamics of uvms
x_next_decoupled, x_nextdd_decoupled, states, u, dt, q_min, q_max, manipulator_p, vehicle_p, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=False)

x_next_decoupled_eval = cs.Function('UVMSnext_decoupled', [states, u, dt, vehicle_p, manipulator_p, base_T ,q_min, q_max], [x_next_decoupled, u_checks, x_nextdd_decoupled])



UVMSnext_use_coupled = cs.Function.if_else('UVMSnext_use_coupled', x_next_coupled_eval, x_next_decoupled_eval)
UVMSnext_use_coupled.save('uvms_forward_dynamics.casadi')

floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found


In [3]:
# # c , cpp or matlab code generation for forward dynamics
# UVMSnext_use_coupled.generate("x_next_eval.c")
# os.system(f"gcc -fPIC -shared x_next_eval.c -o libUVMS_xnext.so")

In [4]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 1.91318, 1.33339, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0.0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981)

UVMSnext_use_coupled(use_coupled, st_x0, u0, alpha.delta_t, blue.sim_p, alpha.sim_p, alpha.base_T0, alpha.joint_min, alpha.joint_max)

(DM([-8.20189e-08, 2.42665e-07, 7.29586e-06, -7.36039e-06, 3.26595e-06, 1.97164e-05, 1.9136, 1.5, 0.992568, 3.16073, -3.06267e-06, 9.01758e-06, -0.0011038, -0.000283182, 0.000110324, 0.000714989, 0.0134925, 1.33818e-07, 0.00362818, -0.00505576]),
 DM([0, 0, 0, 0, 0, 0, 0.0340454, 0, 0.0062739, -0.00505981]),
 DM([1.99165e-05, 4.00955e-05, 1.96424e-05, -0.00055338, 0.000704559, 0.00300836, 0.0236798, 0.0148731, 0.0023032, -0.0977525]))

In [5]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 1.91318, 1.33339, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0.0, 0, 0, 0, 0, 0, 0.0340454, -0.00441695, 0.0062739, -0.00505981)

UVMSnext_use_coupled(not use_coupled, st_x0, u0, alpha.delta_t, blue.sim_p, alpha.sim_p, alpha.base_T0, alpha.joint_min, alpha.joint_max)

(DM([0, 0, 0, 0, 0, 0, 1.9136, 1.5, 0.992568, 3.16073, 0, 0, 0, 0, 0, 0, 0.0134925, 1.33818e-07, 0.00362818, -0.00505576]),
 DM([0, 0, 0, 0, 0, 0, 0.0340454, 0, 0.0062739, -0.00505981]),
 DM([0, 0, 0, 0, 0, 0, 0.0236798, 0.0148731, 0.0023032, -0.0977525]))